# (k)da-PLS 在域自适应中的应用
## Dr. Ramin Nikzad-Langerodi
### Bottleneck Analytics GmbH
info@bottleneck-analytics.com

___
首先，我们加载一些将要使用的模块，包括 di-/da-PLS 类。

In [ ]:
# 加载模块
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from diPLSlib.models import DIPLS as dipls
from diPLSlib.models import KDAPLS as kdapls
from diPLSlib.utils import misc as fct
from sklearn.model_selection import GridSearchCV, cross_val_predict

# 配置中文绑图支持
fct.setup_chinese_plot()

### 模拟数据

让我们创建一些模拟的“源域”和“目标域”数据集，分别包含 N=50 个样本和 p=100 个变量。

In [2]:
n = 50  # 样本数量
p = 100 # 变量数量

为此，我们定义了 3 个高斯函数，其中第一个对应于我们将尝试建模的（分析物）信号，另外两个对应于干扰信号（干扰物）。

源域数据集将仅包含分析物信息以及两个干扰物中**一个**的贡献。

In [ ]:
np.random.seed(10)

# 源域 (分析物 + 1 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物信号
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1 信号
S = np.vstack([S1,S2])

# 分析物和干扰物浓度
Cs = 10*np.random.rand(n,2)

# 光谱
Xs = Cs@S

在目标域中，我们将有来自分析物和**两种**干扰物的贡献。

In [ ]:
# 目标域 (分析物 + 2 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物信号
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1 信号
S3 = fct.gengaus(p, 30, 10, 10, 0) # 干扰物 2 信号
S = np.vstack([S1,S2,S3])

# 分析物和干扰物浓度
Ct = 10*np.random.rand(n,3)

# 光谱
Xt = Ct@S

让我们绘制分析物和干扰物的纯信号以及模拟数据集。

In [ ]:
# 绘制纯信号
plt.figure()

plt.subplot(211)
plt.plot(S1)
plt.plot(S2)
plt.plot(S3)
plt.legend(['分析物','干扰物 1','干扰物 2'])
plt.title('纯信号')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)
plt.axvline(x=30,linestyle=':',color='k',alpha=0.5)

# 源域
plt.subplot(223)
plt.plot(Xs.T, 'b', alpha=0.2)
plt.title('源域')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)

# 目标域
plt.subplot(224)
plt.plot(Xt.T, 'r', alpha=0.2)
plt.title('目标域')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)
plt.axvline(x=30,linestyle=':',color='k',alpha=0.5)
plt.tight_layout()

### di-PLS vs. (k)da-PLS
我们现在将拟合无监督的 di-PLS 和 (k)da-PLS 模型，并比较它们在目标域上的表现。我们首先使用 da-PLS 的原始 (primal) 版本，然后讨论核化 (kernelized) 版本。

请注意，di-PLS 最小化源域和目标域之间的协方差差异，从而隐含地假设相应的数据集遵循正态分布。另一方面，da-PLS 使用非参数方法来对齐潜在空间中的源分布和目标分布，不对底层数据分布做任何假设。

In [ ]:
plt.figure(figsize=(12, 3))

# 源域 PLS 模型
plt.subplot(141)

y = np.expand_dims(Cs[:, 0],1)
m = dipls(A=2, l=0)
l = [0] # 无正则化
m.fit(Xs, y, Xs, Xt)
b_pls = m.b_

# 预测目标域中的分析物
yhat_pls = m.predict(Xt)
plt.scatter(Ct[:, 0], yhat_pls, color='b', edgecolor='k',alpha=0.75)
ax = plt.gca()
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('源域 PLS')

# diPLS 模型
plt.subplot(142)

m_di = dipls(A=2, l=100000)
m_di.fit(Xs, y, Xs, Xt)
b_dipls = m_di.b_

# 预测目标域中的分析物
yhat_dipls = m_di.predict(Xt)
plt.scatter(Ct[:, 0], yhat_dipls, color='r', edgecolor='k',alpha=0.75)
ax = plt.gca()
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('diPLS')

# KDA-PLS 模型
plt.subplot(143)

m_kda = kdapls(A=2, l=1000, target_domain=0, kernel_params={'type': 'primal'})
m_kda.fit(Xs, y, Xs, Xt)
b_kda = m_kda.coef_

# 预测目标域中的分析物
yhat_kda = m_kda.predict(Xt)
plt.scatter(Ct[:, 0], yhat_kda, color='g', edgecolor='k',alpha=0.75)
ax = plt.gca()
ax.set_xlim([0, 10])
ax.set_ylim([0, 10])
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('daPLS')

# 目标域 PLS 模型
plt.subplot(144)

y = np.expand_dims(Ct[:, 0], 1)
m_target = dipls(A=3, l=0)
m_target.fit(Xt, y, Xs, Xt)
b_target = m_target.b_

# 预测目标域中的分析物
yhat_target = m_target.predict(Xt)
plt.scatter(Ct[:, 0], yhat_target, color='m', edgecolor='k',alpha=0.75)
ax = plt.gca()
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('目标域 PLS')

plt.suptitle('目标域预测')
plt.tight_layout()

我们可以看到，在这种情况下，da-PLS 模型明显优于 di-PLS 模型。为了了解原因，我们接下来将检查这 4 个模型对应的回归系数。

In [ ]:
# 绘制回归系数
plt.figure()

plt.plot(b_pls, 'b')
plt.plot(b_dipls, 'r')
plt.plot(b_kda, 'g')
plt.plot(b_target, 'm')
plt.legend(['源域 PLS','diPLS','daPLS','目标域 PLS'])
plt.title('回归系数')
plt.xlabel('X-变量')
plt.ylabel('系数')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)
plt.axvline(x=30,linestyle=':',color='k',alpha=0.5)
plt.tight_layout()

我们可以看到，da-PLS 模型（绿色）的回归系数比 di-PLS 模型（红色）更接近目标域 PLS 模型系数（洋红色）。

接下来，我们将 (k)da-PLS 应用于来自以下地址的三聚氰胺 (Melamine) NIR 数据集：https://github.com/RNL1/Melamine-Dataset

### 三聚氰胺数据

In [ ]:
# 加载并读取数据
url = "https://github.com/RNL1/Melamine-Dataset/blob/master/Melamine_Dataset.pkl?raw=true"
data = pd.read_pickle(url)

wn1 = data['wn1']
wn2 = data['wn2']
w = np.hstack((wn1, wn2))

Xs = np.hstack((data['R862']['X1'], data['R862']['X2']))
Xt = np.hstack((data['R861']['X1'], data['R861']['X2']))

ys = data['R862']['Y']
yt = data['R861']['Y']

#### 超参数调优
我们运行网格搜索来寻找最佳的潜变量 (LVs) 数量和最佳核。请注意，在此步骤中我们仅使用源域数据。

In [ ]:
# 调优 LVs 数量和核参数
param_grid = {'A': np.arange(1, 20),
              'kernel_params': [{'type': 'rbf', 'gamma': 1e-3},
                                {'type': 'linear'},
                                {'type': 'primal'}]}

# 初始化 KDAPLS 模型 
m_kdapls = kdapls(target_domain=0)

# 使用均方根误差作为评分指标调优 LVs 数量
grid_search = GridSearchCV(m_kdapls, param_grid, cv=5, scoring= 'neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(Xs, ys)

# 打印发现的最佳参数和最佳评分
print("发现的最佳参数: ", grid_search.best_params_)

# 绘制 MSE 与 LVs 数量的关系图
opt_A = grid_search.best_params_['A']-1
rmse = -grid_search.cv_results_['mean_test_score']
rmse_std = grid_search.cv_results_['std_test_score']
#plt.errorbar(param_grid['A'], rmse, yerr=rmse_std, fmt='-o', mec='k', label='CV 误差')
rmse_2d = rmse.reshape(len(param_grid["kernel_params"]), len(param_grid["A"]))
plt.imshow(rmse_2d, cmap="viridis", aspect="auto")
plt.colorbar(label="RMSE")
plt.xticks(np.arange(len(param_grid["A"])), param_grid["A"])
kernel_labels = [
    f"{kp['type']} (gamma={kp.get('gamma','-')})" 
    if 'gamma' in kp else kp['type'] 
    for kp in param_grid["kernel_params"]
]
plt.yticks(np.arange(len(kernel_labels)), kernel_labels)
plt.xlabel("LVs 数量")
plt.ylabel("核参数")
plt.title("RMSE 热力图")
plt.gca().axvline(opt_A, color='r', linestyle='--', label='最佳 LVs 数量')
plt.xlabel('LVs 数量')
plt.ylabel('RMSECV')
plt.title('RMSECV (源域) vs. LVs 数量')
plt.legend()
plt.show()

对于这个数据集，使用原始 (primal) da-PLS 模型似乎最合适。核化版本的 da-PLS 在源任务上无法超越原始版本。

接下来我们优化正则化参数，该参数决定了源域和目标域之间的对齐程度。

In [ ]:
# 调优正则化参数
param_grid = {'l': np.logspace(0, 3, 10)}

# 初始化 KDAPLS 模型 
m_kdapls = kdapls(target_domain=0, A=12, kernel_params={'type': 'primal'})

# 使用均方根误差作为评分指标调优 LVs 数量
grid_search = GridSearchCV(m_kdapls, param_grid, cv=5, scoring= 'neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(Xs, ys, **{'xs': Xs, 'xt': Xt}) 

# 打印发现的最佳参数和最佳评分
print("发现的最佳参数: ", grid_search.best_params_)

# 绘制 MSE 与 l 数量的关系图
rmse = -grid_search.cv_results_['mean_test_score']
rmse_std = grid_search.cv_results_['std_test_score']/np.sqrt(5)
plt.plot(param_grid['l'], rmse, 'o-', mec='k', label='CV 误差')
plt.xlabel('正则化参数 l')
plt.ylabel('RMSECV (源域)')

我们希望在不牺牲源域性能的情况下，尽可能选择较大的正则化参数 $l$。这里，我们选择 $l=1000$。

现在我们准备拟合最终模型并评估其在目标域上的表现。

In [ ]:
# 初始化 KDAPLS 模型
m_kdapls = kdapls(target_domain=0, A=12, l=1000, kernel_params={'type': 'primal'})

# 拟合模型
m_kdapls.fit(Xs, ys, **{'xs': Xs, 'xt': Xt})

# 预测目标域
yhat = m_kdapls.predict(Xt)

# 绘制预测值与测量值的关系图
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.scatter(yt, yhat, color='b', edgecolor='k', alpha=0.75)
ax = plt.gca()
ax.set_xlim([-10, 60])
ax.set_ylim([-10, 60])
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('daPLS')
plt.text(0.75, 0.1, f"RMSEP: {fct.rmse(yhat.T, yt.T):.2f}", ha='right', va='center', transform=ax.transAxes)

# 与 PLS 比较
m_kdapls = kdapls(target_domain=0, A=12, l=0, kernel_params={'type': 'primal'})
m_kdapls.fit(Xs, ys)
yhat = m_kdapls.predict(Xt)
plt.subplot(122)
plt.scatter(yt, yhat, color='r', edgecolor='k', alpha=0.75)
ax = plt.gca()
ax.set_xlim([-10, 60])
ax.set_ylim([-10, 60])
ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.grid(axis='x')
plt.title('PLS')
plt.text(0.75, 0.1, f"RMSEP: {fct.rmse(yhat.T, yt.T):.2f}", ha='right', va='center', transform=ax.transAxes)
plt.tight_layout()

当应用于目标域时，da-PLS 明显优于 PLS。接下来我们研究它与 di-PLS 相比的表现如何。

In [ ]:
# diPLS 和 daPLS 的正则化参数
l_da = [1e2, 1e3, 1e10]
l_di = [1e2, 1e3, 1e10]

# 回归系数
b_da = {}
b_di = {}

i = 0
plt.figure(figsize=(12, 6))
for i in range(3):
    plt.subplot(2, 3, i+1)
    m_da = kdapls(A=12, l=l_da[i], target_domain=0, kernel_params={'type': 'primal'})
    m_da.fit(Xs, ys, Xs, Xt)
    yhat_da = m_da.predict(Xt)
    plt.scatter(yt, yhat_da, color='b', edgecolor='k', alpha=0.75)
    ax = plt.gca()
    ax.set_xlim([-20, 80])
    ax.set_ylim([-20, 80])
    ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
    plt.xlabel('测量值')
    plt.ylabel('预测值')
    plt.grid(axis='x')
    plt.title(f"daPLS (l={l_da[i]:.1e})")
    plt.text(0.75, 0.1, f"RMSEP: {fct.rmse(yhat_da.T, yt.T):.2f}", ha='right', va='center', transform=ax.transAxes)
    b_da[l_da[i]] = m_da.coef_

    plt.subplot(2, 3, i+4)
    m_di = dipls(A=12, l=l_di[i])
    m_di.fit(Xs, ys, Xs, Xt)
    yhat_di = m_di.predict(Xt)
    plt.scatter(yt, yhat_di, color='r', edgecolor='k', alpha=0.75)
    ax = plt.gca()
    ax.set_xlim([-20, 80])
    ax.set_ylim([-20, 80])
    ax.plot([0, 1], [0, 1], 'k', transform=ax.transAxes)
    plt.xlabel('测量值')
    plt.ylabel('预测值')
    plt.grid(axis='x')
    plt.title(f"diPLS (l={l_di[i]:.1e})")
    plt.text(0.75, 0.1, f"RMSEP: {fct.rmse(yhat_di.T, yt.T):.2f}", ha='right', va='center', transform=ax.transAxes)
    b_di[l_di[i]] = m_di.b_

plt.tight_layout()

我们可以看到，在正则化参数 $l$ 值较小时，di-PLS 的表现明显更好（左图）。然而，di-PLS 容易产生过拟合，随着我们增加正则化参数，di-PLS 的性能会下降（右图）。另一方面，da-PLS 更加健壮，不会受到过拟合的影响。

为了更清楚地看到这一点，我们绘制相应的回归系数。

In [ ]:
# 绘制回归系数
plt.figure(figsize=(12, 4))
plt.subplot(121)
offset = 0
for i, key in enumerate(b_da):
    plt.plot(b_da[key] + offset, label=f"{key:.1e}")
    offset += 500  # 根据需要调整偏移值
plt.legend([f"{key:.1e}" for key in b_da.keys()])
plt.title('daPLS')
plt.xlabel('X-变量')
plt.ylabel('系数')

# 绘制带偏移的 diPLS 系数
plt.subplot(122)
offset = 0
for i, key in enumerate(b_di):
    plt.plot(b_di[key] + offset, label=f"{key:.1e}")
    offset += 500  # 根据需要调整偏移值
plt.legend([f"{key:.1e}" for key in b_di.keys()])
plt.title('diPLS')
plt.xlabel('X-变量')
plt.ylabel('系数')

plt.tight_layout()

我们可以看到，随着正则化参数的增加，di-PLS 系数变得越来越起伏不平。

### 多个目标域

di-PLS 和 da-PLS 之间另一个值得进行的比较是当我们有多个目标域时。

In [ ]:
### 源域 (分析物 + 1 种干扰物)
n = 50  # 样本数量
p = 100 # 变量数量

# 生成信号
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物
S = np.vstack([S1,S2])

# 分析物浓度
Cs = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Cs@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 源域光谱加上噪声
Xs = X# + noise

# 目标域 1 (分析物 + 2 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1
S3 = fct.gengaus(p, 30, 10, 10, 0) # 干扰物 2
S = np.vstack([S1,S2,S3])

# 分析物浓度
Ct1 = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Ct1@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 目标域光谱加上噪声
Xt1 = X# + noise

# 目标域 2 (分析物 + 3 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1
S3 = fct.gengaus(p, 30, 10, 10, 0) # 干扰物 2
S4 = fct.gengaus(p, 75, 75, 150, 0) # 干扰物 3
S = np.vstack([S1, S2, S3, S4])

# 分析物浓度
Ct2 = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Ct2@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 目标域光谱加上噪声
Xt2 = X# + noise

plt.figure(figsize=(12,5))
plt.subplot(211)
plt.plot(S1)
plt.plot(S2)
plt.plot(S3)
plt.plot(S4)
plt.legend(['分析物','干扰物 1','干扰物 2', '干扰物 3'])
plt.title('纯信号')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)
plt.axvline(x=30,linestyle=':',color='k',alpha=0.5)


plt.subplot(234)
plt.plot(Xs.T, 'b', alpha=0.2)
plt.title('源域')
plt.xlabel('X-变量')
plt.ylabel('信号')

plt.subplot(235)
plt.plot(Xt1.T, 'r', alpha=0.2)
plt.title('目标域 1')
plt.xlabel('X-变量')
plt.ylabel('信号')

plt.subplot(236)
plt.plot(Xt2.T, 'g', alpha=0.2)
plt.title('目标域 2')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.tight_layout()
plt.show()

我们将拟合一个具有 2 个 LVs 的 da-PLS 模型，并使 $l$ 足够大以对齐源域和目标域。

In [ ]:
# 准备工作
nr_comp = 2                         # 组件数量
l = 1e9   # 正则化参数                           
target_domains = [Xt1, Xt2]         # 目标域
ys = np.expand_dims(Cs[:, 0],1)     # 源域浓度
yt1 = np.expand_dims(Ct1[:, 0],1)   # 目标域 1 浓度
yt2 = np.expand_dims(Ct2[:, 0],1)   # 目标域 2 浓度

# daPLS 模型
m_kdapls = kdapls(A=nr_comp, l=l, kernel_params={'type': 'primal'})
m_kdapls.fit(Xs, ys, Xs, target_domains)
b_dapls = m_kdapls.coef_
yhat_daplsT1 = m_kdapls.predict(Xt1)
yhat_daplsT2 = m_kdapls.predict(Xt2)

error_diplsT1 = fct.rmse(yt1, yhat_daplsT1)
error_diplsT2 = fct.rmse(yt2, yhat_daplsT2)
print(f"RMSEP 目标域 1: {error_diplsT1:.2f}")
print(f"RMSEP 目标域 2: {error_diplsT2:.2f}")

min_ = -5
max_ = 15

# 绘制
plt.figure(figsize=(9, 4))
plt.subplot(121)
plt.scatter(yt1, yhat_daplsT1, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 1 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')

plt.subplot(122)
plt.scatter(yt2, yhat_daplsT2, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 2 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.suptitle('da-PLS')
plt.tight_layout()

如我们所见，生成的（单个 da-PLS）模型在两个目标域上都表现良好。另一方面，对于 mdi-PLS，我们需要指定要为哪个目标域拟合模型（参见 `demo_mdiPLS.ipynb`）。